<a href="https://colab.research.google.com/github/sophiamaria05/IC_MDA/blob/update-code-to-first-official-version/Tokenize_xml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import html

##Importing the libraries and installing dependancies

In [ ]:
import os

if os.environ.get("TOKENIZE_PIP_ALREADY_RUN") != "1":
  print("\n\nRunning Pip commands...\n\n")

  import subprocess
  import sys

  subprocess.check_call([sys.executable, "-m", "pip", "install", "-U", "h5py"])
  subprocess.check_call([sys.executable, "-m", "pip", "install", "typing-extensions"])
  subprocess.check_call([sys.executable, "-m", "pip", "install", "wheel"])
  os.environ["TOKENIZE_PIP_ALREADY_RUN"] = "1"
else:
    print("\n\nPip commands were already executed!\n\n")

In [ ]:
import xml.etree.ElementTree as ET
from xml.dom.minidom import parseString

##Defining the functions

In [ ]:
def count_nmod(dep):
  nmod = 0
  for i in range(0, len(dep)):
    if dep[i]=="nmod":
      nmod+=1
  return nmod

In [ ]:
def tokenize_phrase(phrase):
  try:
    nodes = phrase.text.split()

    nodes = [item.replace(",", "") for item in nodes]
    remove_nodes = {"e", "o", "a", ",", "de", "do", "da", "para"}
    nodes[:] = [item for item in nodes if item not in remove_nodes]

    if len(nodes) < 3:
      print("INVALID PHRASE: "+phrase.text)
      return False
  except Exception as e:
    print("PHRASE NULL")
    return False

  ET.SubElement(phrase, "noun").text = nodes[0].title()
  ET.SubElement(phrase, "verb").text = nodes[1].title()

  if (nodes[1].title() == "Informa") or (nodes[1].title() == "Informar"):
    ET.SubElement(phrase, "msg").text = phrase.text.split(nodes[1], 1)[1]
    return True

  ET.SubElement(phrase, "object").text = nodes[2].title()

  nodes = [item for item in nodes if item not in set(nodes[:2])][1:]
  for i in range(0, len(nodes)):
    complement = ET.SubElement(phrase, "complement")
    complement.text = nodes[i].title()
    complement.set('id', str(i))

  return True

In [ ]:
def get_xml_files(xml_file_folder):
  """
    Gets the folder path where the xml file with predicted phrases.
    Returns a List[xml_files].
  """
  return [file for file in os.listdir(xml_file_folder) if file.endswith('.xml')]

In [ ]:
def tokenize(file_path, write_name):
  xml = ET.parse(file_path, parser=ET.XMLParser(encoding='utf-8'))
  xml_fix = ET.tostring(xml.getroot(), encoding='utf-8').decode('utf-8')
  root = ET.fromstring(xml_fix)
  xml_string = parseString(xml_fix)

  for use_case in root:
    for flow in use_case:
      for phrase in flow:
        tokenize_phrase(phrase)

  tree = ET.ElementTree(root)
  tree.write(write_name)
  print(">>", file_path, "<< TOKENIZED AND SAVED ON >>", write_name, "<<")
  print(html.unescape(parseString(ET.tostring(tree.getroot())).toprettyxml(indent='\t', newl='\n', )))

##Runing the tokenization

In [ ]:
folder = os.environ.get("TO_BE_TOKENIZED_DIR")
if folder == None or folder == "":
  folder = "/content/"

save_folder = os.environ.get("TOKENIZED_XML_DIR")
if save_folder == None or save_folder == "":
  save_folder = "/content/tokenized/"
os.makedirs(save_folder, exist_ok=True)

files = get_xml_files(folder)
for file_name in files:
  tokenize(folder+"/"+file_name, save_folder+"tokenized_phrases ("+file_name.split('.', 1)[0]+").xml")